In [163]:
from deap import creator, base, tools, algorithms
import random
import numpy

In [170]:
with open("11.txt") as f:
    line1 = f.readline().rstrip() # сразу удаляем перенос строки в конце
    MAX_WEIGHT = int(line1.split(' ')[0]) # грузоподъемность
    MAX_VOLUME = int(line1.split(' ')[1]) # вместимость
    items = {}
    for num, line in enumerate(f):
        items[num] = tuple(map(float, line.rstrip().split(' ')))
    
IND_INIT_SIZE = 5 # начальное количество предметов особи
MAX_ITEM = 50   # максимальное количество предметов в рюкзаке (в особи)
NBR_ITEMS = 20 # количество предметов

In [171]:
# Инициализируем популяцию и существ

# функция приспособленности: минимизация параметра 1 при максимизации параметра 2
creator.create("Fitness", base.Fitness, weights=(-1.0, -1.0, 1.0))
# особь: неупорядоченный набор признаков
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()
# конструирование признака особи: случайное число от 1 до 19
toolbox.register("attr_item", random.randrange, NBR_ITEMS)
# конструирование особи: помещаем в особь атрибуты числом 5
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
# конструирование популяции: список особей
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [174]:
# Подсчет суммарного веса и стоимости
def evalKnapsack(individual):
    weight = 0.0
    volume = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        volume += items[item][1]
        value += items[item][2]        
    if (len(individual) > MAX_ITEM) or (weight > MAX_WEIGHT) or (volume > MAX_VOLUME):
        return 10000, 10000, 0,             # Ensure overweighted bags are dominated
    return weight, volume, value

# Оператор наследования, определяет детей двух множеств: первый как пересечение, второй как разность
def cxSet(ind1, ind2):
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

# Оператор мутации, рандомно добавляет и удаляет элементы
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

# Регистрируем операции
toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [197]:
def main():
    random.seed(64)
    NGEN = 60   # Количество поколений
    MU = 50    # Количество особей, переходящих в следующее поколение
    LAMBDA = 100 # Количество потомков, появляющихся к следующему поколению
    CXPB = 0.7  # Вероятность появления потомка кроссовером
    MUTPB = 0.3 # Вероятность появления потомка мутацией
    
    # популяция: список особей (под капотом каждая особь - множество признаков-чисел от 0 до 20)
    pop = toolbox.population(n=MU)
    # передний зал славы, содержит особей, наилучших из тех, которые никогда не доминировали. отсортирован по ухудшению (первый - самый лучший и т.д.)
    hof = tools.ParetoFront()
    # статистика списка всех особей поколения, узнаем среднее, стандартное отклонение, минимум, максимум
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    # Эволюционная стратегия "Мю плюс лямбда"
    pop100 = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats, halloffame=hof, verbose=0)
    
    best = pop100[0][0]
    for ind in pop100[0]:
        if evalKnapsack(ind)[2] > evalKnapsack(best)[2]:
            best = ind
    weight, volume, value = evalKnapsack(best)
    print("After", NGEN, "generations best individual is", best)
    print("Weight", weight, "of max", MAX_WEIGHT)
    print("Volume", round(volume, 1), "of max", MAX_VOLUME)
    print("Value", value)
    

main()

After 60 generations best individual is Individual({0, 1, 2, 4, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19})
Weight 12847.0 of max 13000
Volume 10.7 of max 12
Value 4036.0
